<a href="https://colab.research.google.com/github/Sheela-Nathan/Git-initialisation/blob/main/Retention_data_for_mega_and_micro_stores_ipynb_from_first_transaction_date.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q pygsheets

In [ ]:
!pip install openpyxl

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd
import math
import numpy as np
import datetime as dt
import calendar
pd.options.mode.chained_assignment = None  # default='warn'

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta,date
from dateutil import rrule

import pygsheets
from pygsheets import WorksheetNotFound
import google.auth
from google.colab import auth
from google.colab import files
import io


In [ ]:
auth.authenticate_user()
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [ ]:
# Function for reading google sheet contents
def open_sheet(key, sheet_name):

  sheet = gc.open_by_key(key)
  open_sheet = sheet.worksheet_by_title(sheet_name)
  df = open_sheet.get_as_df()

  return df

# Function for exporting dataframe to google sheets
def push_to_sheets(df, key, sheet_name, start):

  sheet = gc.open_by_key(key)
  try:
    out_dataset = sheet.worksheet_by_title(sheet_name)
  except WorksheetNotFound:
    out_dataset = sheet.add_worksheet(title = sheet_name,rows=10000,cols=100)
  out_dataset.clear()
  out_dataset.set_dataframe(df, start = start, nan = '')

# Function for updating a single value in a google sheet
def update_value(value, key, sheet_name, loc):
    sheet = gc.open_by_key(key)
    out_dataset = sheet.worksheet_by_title(sheet_name)
    out_dataset.update_value(loc,value)

In [ ]:
pack_cl = {
    "user": "packuser",
    "password": "p5cKw0Rds",
    "server_host": "159.223.60.204",
    "port": "5432",
    "database":"packdata"
}
pack_engine= create_engine('postgresql://{user}:{password}@{server_host}:{port}/{database}'.format(**pack_cl))

In [ ]:
conn=pack_engine.connect()

In [ ]:
query='''with megastores as(
select * from store
where store_type in ('super_store','packworks')
and deleted_at is null and (status IS NULL OR status = 'approved'))
SELECT store_id,customer_id,created_at,sales_entry_id
FROM sales_entry
WHERE store_id::text in (select store_id::text from megastores)
and created_at between '2020-01-01' and '2023-09-30'
and deleted_at is null;'''


In [ ]:
df= pd.read_sql_query(sql=text(query),con=pack_engine.connect())

In [ ]:
df.head()

,store_id,customer_id,created_at,sales_entry_id
0,5f73ac3f-216b-4ac2-b5ab-a37f99ac02a3,None,2022-06-28 14:03:31,f26d8885-306b-4fae-8c68-c3b04827acdb
1,e51db60c-051a-4d4b-b1b5-ff542adab60f,None,2022-06-28 12:29:37,f25f8cce-7702-4601-9ead-5099dbfed15b
2,d7cdeb3a-b83d-40a2-8ef7-1731b74b1883,None,2022-06-28 11:53:26,f27fd58d-b090-4d56-a373-c3bf9fbd7afd
3,4b4727df-6af9-42e8-96b2-27369bc5da7c,b25237a5-2303-4482-8cdf-fc678df17f1c,2022-06-28 11:29:00,f266ba4c-f9d6-46d3-97c2-63bca05044c8
4,3b725904-b7d3-43f1-a3f9-1eaf008a010b,None,2022-06-28 11:18:46,f2749630-ac65-4f4f-bf1e-efcc1da7d3f7


In [ ]:
df_mega=df[['store_id','created_at']]

In [ ]:
df_mega.head()

,store_id,created_at
0,5f73ac3f-216b-4ac2-b5ab-a37f99ac02a3,2022-06-28 14:03:31
1,e51db60c-051a-4d4b-b1b5-ff542adab60f,2022-06-28 12:29:37
2,d7cdeb3a-b83d-40a2-8ef7-1731b74b1883,2022-06-28 11:53:26
3,4b4727df-6af9-42e8-96b2-27369bc5da7c,2022-06-28 11:29:00
4,3b725904-b7d3-43f1-a3f9-1eaf008a010b,2022-06-28 11:18:46


In [ ]:
df_micro=df[['customer_id','created_at']]

In [ ]:
df_micro=df_micro.dropna(subset=['customer_id'])
df_micro

,customer_id,created_at
3,b25237a5-2303-4482-8cdf-fc678df17f1c,2022-06-28 11:29:00
5,cf1393b2-2a24-4ead-846f-fba6237453d5,2022-06-28 10:24:50
7,80c6f496-b872-4a09-b79e-6bdc976d6e4f,2022-06-28 09:52:48
12,47324457-8953-4524-b30f-955dc3e8d881,2022-05-28 09:51:26
19,6399c1a3-94df-46df-b55d-1986c84be921,2022-06-28 16:35:00
...,...,...
20020685,dfdf438f-e591-4710-a284-434e2e0c3cb9,2023-08-31 10:37:29
20020688,7b623993-b4f3-4e1e-a988-06a3c3163de3,2023-08-30 22:49:30
20020690,467c0c64-a5b2-4cc2-8d28-111f09dcee7e,2023-08-30 13:01:55
20020691,ea04f436-d613-4fe6-97fc-2a006fbe423c,2023-08-30 12:39:35



#Megastore cohort analysis

In [ ]:
#function for getting the active month
def getting_months(m):
    return dt.datetime(m.year, m.month,1)

#using the above function
df_mega['Active_Month'] = df_mega['created_at'].apply(getting_months)

In [ ]:
df_micro['Active_Month'] = df_micro['created_at'].apply(getting_months)

# Indexing the Column By First Month of Activity


In [ ]:
#indexing a column for the first month activity of the megastores
df_mega['Cohort-Month'] =  df_mega.groupby('store_id')['Active_Month'].transform('min')
df_mega.head(30)

,store_id,created_at,Active_Month,Cohort-Month
0,5f73ac3f-216b-4ac2-b5ab-a37f99ac02a3,2022-06-28 14:03:31,2022-06-01,2021-09-01
1,e51db60c-051a-4d4b-b1b5-ff542adab60f,2022-06-28 12:29:37,2022-06-01,2020-01-01
2,d7cdeb3a-b83d-40a2-8ef7-1731b74b1883,2022-06-28 11:53:26,2022-06-01,2020-01-01
3,4b4727df-6af9-42e8-96b2-27369bc5da7c,2022-06-28 11:29:00,2022-06-01,2021-03-01
4,3b725904-b7d3-43f1-a3f9-1eaf008a010b,2022-06-28 11:18:46,2022-06-01,2020-12-01
5,13e29fd9-1851-4ed8-a27e-790a4bc294db,2022-06-28 10:24:50,2022-06-01,2020-11-01
6,137c85e7-0c18-4b11-b6fc-f58e43c4f433,2022-06-28 10:05:33,2022-06-01,2020-09-01
7,01258973-987d-46fb-baa0-9711f600ee81,2022-06-28 09:52:48,2022-06-01,2020-01-01
8,5b53ce9d-e3a5-49a1-891a-bf933a5d26c9,2022-06-28 07:51:10,2022-06-01,2021-05-01
9,bef452b7-4434-49b2-b601-2f38b6247e5f,2022-06-28 07:48:25,2022-06-01,2021-02-01


In [ ]:
#indexing a column for the first month activity of the microstores
df_micro['Cohort-Month'] =  df_micro.groupby('customer_id')['Active_Month'].transform('min')
df_micro.head(30)

,customer_id,created_at,Active_Month,Cohort-Month
3,b25237a5-2303-4482-8cdf-fc678df17f1c,2022-06-28 11:29:00,2022-06-01,2021-05-01
5,cf1393b2-2a24-4ead-846f-fba6237453d5,2022-06-28 10:24:50,2022-06-01,2021-05-01
7,80c6f496-b872-4a09-b79e-6bdc976d6e4f,2022-06-28 09:52:48,2022-06-01,2020-08-01
12,47324457-8953-4524-b30f-955dc3e8d881,2022-05-28 09:51:26,2022-05-01,2021-06-01
19,6399c1a3-94df-46df-b55d-1986c84be921,2022-06-28 16:35:00,2022-06-01,2020-08-01
30,4a833e3a-3524-4e71-8125-f10d1f05eae8,2023-08-31 16:34:52,2023-08-01,2021-06-01
32,5e1f97ec-5dcd-48fe-860c-c88bfac68ead,2023-08-31 15:43:06,2023-08-01,2023-06-01
37,4e67918b-2ca2-41b0-94cd-9bff11f80df8,2023-08-31 13:49:34,2023-08-01,2021-08-01
39,a0526d72-2a2f-476e-8dc6-0bb815494d05,2023-08-31 13:07:09,2023-08-01,2021-06-01
41,c3dd124a-d537-4a94-8df5-a0eef20dbf67,2023-08-31 10:53:38,2023-08-01,2021-11-01


In [ ]:
pivot_table = df_mega.pivot_table(index='Cohort-Month', columns='Active_Month', values='store_id', aggfunc=pd.Series.nunique)


In [ ]:
pivot_table_micro = df_micro.pivot_table(index='Cohort-Month', columns='Active_Month', values='customer_id', aggfunc=pd.Series.nunique)

In [ ]:
pivot_table = pivot_table.fillna(0)

In [ ]:
pivot_table_micro=pivot_table_micro.fillna(0)

In [ ]:
# Drop rows and columns with all zeros
pivot_table = pivot_table.loc[(pivot_table != 0).any(axis=1), (pivot_table != 0).any(axis=0)]

In [ ]:
pivot_table_micro = pivot_table_micro.loc[(pivot_table_micro != 0).any(axis=1), (pivot_table_micro != 0).any(axis=0)]

In [ ]:
# Convert count values to integers
pivot_table = pivot_table.fillna(0).astype(int)

In [ ]:
pivot_table_micro= pivot_table_micro.fillna(0).astype(int)

In [ ]:
pivot_table = pivot_table.replace(0, '')

In [ ]:
pivot_table_micro=pivot_table_micro.replace(0, '')

# Initially i changed to string format and then to date time format, so while runing use
pivot_table.index = pivot_table.index.strftime('%B %Y')
pivot_table.columns = pivot_table.columns.strftime('%B %Y') alone and reset the index

In [ ]:
# Assuming you have a DataFrame pivot_table with the index formatted as month name and year
pivot_table.index = pd.to_datetime(pivot_table.index, format='%Y-%m-%d').strftime('%B %Y')

pivot_table.columns = pd.to_datetime(pivot_table.columns, format='%Y-%m-%d').strftime('%B %Y')


# Reset the index
pivot_table = pivot_table.reset_index()


In [ ]:
pivot_table_micro.index = pd.to_datetime(pivot_table_micro.index, format='%Y-%m-%d').strftime('%B %Y')

pivot_table_micro.columns = pd.to_datetime(pivot_table_micro.columns, format='%Y-%m-%d').strftime('%B %Y')


# Reset the index
pivot_table_micro = pivot_table_micro.reset_index()


In [ ]:
pivot_table

Active_Month,Cohort-Month,January 2020,February 2020,March 2020,April 2020,May 2020,June 2020,July 2020,August 2020,September 2020,...,December 2022,January 2023,February 2023,March 2023,April 2023,May 2023,June 2023,July 2023,August 2023,September 2023
0,January 2020,117,76,67,51,47,79,80,80,75,...,99,98,95,95,92,93,87,90,89,91
1,February 2020,,24,9,5,7,11,7,12,9,...,23,23,23,23,22,23,20,21,21,21
2,March 2020,,,9,5,4,5,6,5,4,...,7,7,7,7,7,7,7,7,7,7
3,May 2020,,,,,3,3,3,2,2,...,1,2,2,2,1,2,2,2,2,2
4,June 2020,,,,,,41,28,21,16,...,35,35,36,34,35,35,32,32,31,31
5,July 2020,,,,,,,109,74,62,...,89,83,83,79,78,80,71,75,80,76
6,August 2020,,,,,,,,125,94,...,102,101,99,97,97,95,86,90,91,86
7,September 2020,,,,,,,,,150,...,115,113,112,110,109,106,91,97,98,94
8,October 2020,,,,,,,,,,...,131,130,123,122,123,122,115,114,114,111
9,November 2020,,,,,,,,,,...,67,64,58,56,55,55,45,47,53,47


In [ ]:
pivot_table_micro

Active_Month,Cohort-Month,January 2020,February 2020,March 2020,April 2020,May 2020,June 2020,July 2020,August 2020,September 2020,...,December 2022,January 2023,February 2023,March 2023,April 2023,May 2023,June 2023,July 2023,August 2023,September 2023
0,January 2020,84,55,46,46,47,49,49,53,52,...,74,73,68,74,67,67,65,54,60,40
1,February 2020,,35,7,3,4,4,5,6,4,...,27,27,21,24,24,24,22,24,24,19
2,March 2020,,,13,4,3,4,5,6,4,...,12,12,11,10,11,8,8,9,9,8
3,April 2020,,,,2,1,,1,1,1,...,2,2,2,2,2,2,1,2,1,1
4,May 2020,,,,,3,1,,2,1,...,3,3,3,3,3,2,2,3,1,1
5,June 2020,,,,,,41,11,12,6,...,30,29,30,28,25,21,13,14,14,11
6,July 2020,,,,,,,383,80,74,...,212,236,202,207,207,207,164,199,214,168
7,August 2020,,,,,,,,238,78,...,184,176,172,174,164,162,155,160,166,157
8,September 2020,,,,,,,,,329,...,228,215,204,197,202,206,205,201,189,184
9,October 2020,,,,,,,,,,...,256,234,240,235,226,234,198,199,200,186


In [ ]:
#push_to_sheets(df =pivot_table , key = "1uvaCxNE7a2o476KQ_g5AJHdmhZpXGWqSgLj_7Bc8xbU", sheet_name = 'Retention of megastores', start = 'A2')

In [ ]:
#push_to_sheets(df =pivot_table_micro , key = "1uvaCxNE7a2o476KQ_g5AJHdmhZpXGWqSgLj_7Bc8xbU", sheet_name = 'Retention of microstores', start = 'A2')